In [1]:
%run ../Python_files/util.py

No dicts found; please check load_dicts...


In [2]:
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix
import json

In [3]:
with open('../temp_files/route_dict_journal.json', 'r') as json_file:
    route_dict = json.load(json_file)

In [4]:
route_dict['1']['link_idx_list']

[92, 81, 84, 112]

In [5]:
route_dict

{u'13357': {u'desti': 31,
  u'length_of_route': 49.41366182901165,
  u'link_idx_list': [66, 78, 88, 96, 125, 128, 230, 244, 247],
  u'node_list': [19, 22, 23, 24, 33, 32, 60, 63, 65, 64],
  u'origi': 8},
 u'13356': {u'desti': 31,
  u'length_of_route': 48.4937433549711,
  u'link_idx_list': [66, 78, 90, 122, 232, 247],
  u'node_list': [19, 22, 23, 31, 60, 65, 64],
  u'origi': 8},
 u'11542': {u'desti': 1,
  u'length_of_route': 52.158106030616565,
  u'link_idx_list': [116, 91, 79, 59, 54, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 16, 17, 6, 3, 2],
  u'origi': 14},
 u'11543': {u'desti': 1,
  u'length_of_route': 52.69422945554072,
  u'link_idx_list': [116, 91, 79, 61, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 17, 6, 3, 2],
  u'origi': 14},
 u'11540': {u'desti': 25,
  u'length_of_route': 5.334022101781262,
  u'link_idx_list': [254],
  u'node_list': [69, 70],
  u'origi': 33},
 u'11541': {u'desti': 1,
  u'length_of_route': 46.93644430865088,
  u'link_idx_list': [71, 43, 38, 31, 21, 9, 7],
 

### create route_choice_probability_matrix $\bf{P} = \left[p_{ir}\right]$ (\# of OD pairs $\times$ \# of routes)

In [6]:
od_nodes_dict = zload('../temp_files/od_nodes_dict_journal.pkz')

In [7]:
od_nodes_dict

{0: {1},
 1: {2, 4, 5},
 2: {3},
 3: {6, 8},
 4: {7, 9},
 5: {10, 11},
 6: {12, 15},
 7: {13, 14},
 8: {16, 17, 18, 19},
 9: {20},
 10: {21},
 11: {22},
 12: {23, 24, 25, 26, 27, 28},
 13: {29},
 14: {30, 31, 32, 33, 34},
 15: {35, 71},
 16: {36},
 17: {37, 38, 39, 40, 41},
 18: {42},
 19: {43, 44, 45, 46, 47},
 20: {48, 74},
 21: {49, 73},
 22: {50, 51},
 23: {52},
 24: {53},
 25: {54, 68, 70},
 26: {55, 56},
 27: {57},
 28: {58},
 29: {59, 60, 72},
 30: {61},
 31: {62, 63, 64, 65, 66},
 32: {67},
 33: {69}}

In [8]:
# od_pair_label_dict = {}

# idx = 0
# for origi in range(len(od_nodes_dict)):
#     for desti in range(len(od_nodes_dict)):
#         if origi != desti:
#             od_pair_label_dict[idx] = [origi, desti]
#             idx += 1
            
# zdump(od_pair_label_dict, '../temp_files/od_pair_label_dict_journal.pkz')

In [9]:
od_list = []
od_list.append(11) # add Boston downtown
for key in od_nodes_dict.keys():
    if len(od_nodes_dict[key]) >= 2:
        od_list.append(key)

In [10]:
od_list

[11, 1, 3, 4, 5, 6, 7, 8, 12, 14, 15, 17, 19, 20, 21, 22, 25, 26, 29, 31]

In [11]:
# od_pair_label_dict = {}

# idx = 0
# for origi in range(len(od_nodes_dict)):
#     for desti in range(len(od_nodes_dict)):
#         if origi != desti:
#             od_pair_label_dict[idx] = [origi, desti]
#             idx += 1
            
# zdump(od_pair_label_dict, '../temp_files/od_pair_label_dict_journal.pkz')

In [12]:
od_pair_label_dict = {}

idx = 0
for origi in od_list:
    for desti in od_list:
        if origi != desti:
            od_pair_label_dict[idx] = [origi, desti]
            idx += 1
            
zdump(od_pair_label_dict, '../temp_files/od_pair_label_dict_journal.pkz')

In [13]:
od_pair_label_dict

{0: [11, 1],
 1: [11, 3],
 2: [11, 4],
 3: [11, 5],
 4: [11, 6],
 5: [11, 7],
 6: [11, 8],
 7: [11, 12],
 8: [11, 14],
 9: [11, 15],
 10: [11, 17],
 11: [11, 19],
 12: [11, 20],
 13: [11, 21],
 14: [11, 22],
 15: [11, 25],
 16: [11, 26],
 17: [11, 29],
 18: [11, 31],
 19: [1, 11],
 20: [1, 3],
 21: [1, 4],
 22: [1, 5],
 23: [1, 6],
 24: [1, 7],
 25: [1, 8],
 26: [1, 12],
 27: [1, 14],
 28: [1, 15],
 29: [1, 17],
 30: [1, 19],
 31: [1, 20],
 32: [1, 21],
 33: [1, 22],
 34: [1, 25],
 35: [1, 26],
 36: [1, 29],
 37: [1, 31],
 38: [3, 11],
 39: [3, 1],
 40: [3, 4],
 41: [3, 5],
 42: [3, 6],
 43: [3, 7],
 44: [3, 8],
 45: [3, 12],
 46: [3, 14],
 47: [3, 15],
 48: [3, 17],
 49: [3, 19],
 50: [3, 20],
 51: [3, 21],
 52: [3, 22],
 53: [3, 25],
 54: [3, 26],
 55: [3, 29],
 56: [3, 31],
 57: [4, 11],
 58: [4, 1],
 59: [4, 3],
 60: [4, 5],
 61: [4, 6],
 62: [4, 7],
 63: [4, 8],
 64: [4, 12],
 65: [4, 14],
 66: [4, 15],
 67: [4, 17],
 68: [4, 19],
 69: [4, 20],
 70: [4, 21],
 71: [4, 22],
 72: [4,

In [14]:
# number of OD pairs
number_of_od_pairs = len(od_pair_label_dict)

In [15]:
od_pair_label_dict[0]

[11, 1]

In [16]:
od_pair_routes_dict = {}
for row in range(number_of_od_pairs):
    routes = []
    for column in range(len(route_dict)):
        if route_dict[str(column)]['origi'] == od_pair_label_dict[row][0] and \
        route_dict[str(column)]['desti'] == od_pair_label_dict[row][1]:
            routes.append(column)
            
    route_length_list = [route_dict[str(column)]['length_of_route'] for column in routes]
    
    refined_routes = []

    number_of_routes_per_OD_pair = 10
    if len(routes) <= number_of_routes_per_OD_pair:
        refined_routes = routes
    else:
        refined_idx_list = [np.argsort(route_length_list)[i] for i in range(number_of_routes_per_OD_pair)]
        refined_routes = [routes[idx] for idx in refined_idx_list]
        
    od_pair_routes_dict[row] = refined_routes

In [17]:
od_pair_routes_dict

{0: [4809, 4810, 4811, 5361, 5362, 5363, 10295, 10296, 10297],
 1: [10501, 10502, 10503, 12012, 12013, 12014],
 2: [12081, 12082, 12083, 13068, 13069, 13070],
 3: [11994, 11995, 11996, 11997, 11998, 11999],
 4: [1882, 1883, 1884, 3608, 3609, 3610],
 5: [6991, 6992, 6993, 11982, 11983, 11984],
 6: [1909, 1912, 1913, 2937, 1910, 2938, 2940, 1911, 2939, 2941],
 7: [2934, 2931, 9083, 2955, 2958, 2959, 2956, 3006, 9084, 2960],
 8: [15475, 15476, 15469, 15477, 14709, 15478, 15472, 15473, 15479, 15474],
 9: [8746, 8747, 8748, 15481, 15482, 15483],
 10: [5418, 5427, 5442, 5428, 5443, 5419, 15442, 13571, 15443, 13572],
 11: [5481, 5482, 5483, 7587, 7588, 5475, 7589, 6312, 5476, 6313],
 12: [880, 881, 882, 2756, 2757, 2758],
 13: [2759, 2760, 2761, 6037, 6038, 6039],
 14: [6031, 6032, 6033, 6040, 6041, 6042],
 15: [2744, 2745, 2746, 6052, 6053, 6054, 7141, 7142, 7143],
 16: [6043, 6044, 6045, 15454, 15455, 15456],
 17: [2750, 2751, 2752, 13511, 13512, 13513, 13520, 13521, 13522],
 18: [12555, 13

In [18]:
# re-label routes
new_route_label_dict = {}
route_idx = 0
for od_pair_idx in range(len(od_pair_routes_dict)):
    for route in od_pair_routes_dict[od_pair_idx]:
        new_route_label_dict[route_idx] = route
        route_idx += 1

In [19]:
new_route_label_dict

{0: 4809,
 1: 4810,
 2: 4811,
 3: 5361,
 4: 5362,
 5: 5363,
 6: 10295,
 7: 10296,
 8: 10297,
 9: 10501,
 10: 10502,
 11: 10503,
 12: 12012,
 13: 12013,
 14: 12014,
 15: 12081,
 16: 12082,
 17: 12083,
 18: 13068,
 19: 13069,
 20: 13070,
 21: 11994,
 22: 11995,
 23: 11996,
 24: 11997,
 25: 11998,
 26: 11999,
 27: 1882,
 28: 1883,
 29: 1884,
 30: 3608,
 31: 3609,
 32: 3610,
 33: 6991,
 34: 6992,
 35: 6993,
 36: 11982,
 37: 11983,
 38: 11984,
 39: 1909,
 40: 1912,
 41: 1913,
 42: 2937,
 43: 1910,
 44: 2938,
 45: 2940,
 46: 1911,
 47: 2939,
 48: 2941,
 49: 2934,
 50: 2931,
 51: 9083,
 52: 2955,
 53: 2958,
 54: 2959,
 55: 2956,
 56: 3006,
 57: 9084,
 58: 2960,
 59: 15475,
 60: 15476,
 61: 15469,
 62: 15477,
 63: 14709,
 64: 15478,
 65: 15472,
 66: 15473,
 67: 15479,
 68: 15474,
 69: 8746,
 70: 8747,
 71: 8748,
 72: 15481,
 73: 15482,
 74: 15483,
 75: 5418,
 76: 5427,
 77: 5442,
 78: 5428,
 79: 5443,
 80: 5419,
 81: 15442,
 82: 13571,
 83: 15443,
 84: 13572,
 85: 5481,
 86: 5482,
 87: 5483,
 

In [20]:
route_dict

{u'13357': {u'desti': 31,
  u'length_of_route': 49.41366182901165,
  u'link_idx_list': [66, 78, 88, 96, 125, 128, 230, 244, 247],
  u'node_list': [19, 22, 23, 24, 33, 32, 60, 63, 65, 64],
  u'origi': 8},
 u'13356': {u'desti': 31,
  u'length_of_route': 48.4937433549711,
  u'link_idx_list': [66, 78, 90, 122, 232, 247],
  u'node_list': [19, 22, 23, 31, 60, 65, 64],
  u'origi': 8},
 u'11542': {u'desti': 1,
  u'length_of_route': 52.158106030616565,
  u'link_idx_list': [116, 91, 79, 59, 54, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 16, 17, 6, 3, 2],
  u'origi': 14},
 u'11543': {u'desti': 1,
  u'length_of_route': 52.69422945554072,
  u'link_idx_list': [116, 91, 79, 61, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 17, 6, 3, 2],
  u'origi': 14},
 u'11540': {u'desti': 25,
  u'length_of_route': 5.334022101781262,
  u'link_idx_list': [254],
  u'node_list': [69, 70],
  u'origi': 33},
 u'11541': {u'desti': 1,
  u'length_of_route': 46.93644430865088,
  u'link_idx_list': [71, 43, 38, 31, 21, 9, 7],
 

In [21]:
new_route_dict = {}
for key in new_route_label_dict.keys():
    new_route_dict[key] = route_dict[str(new_route_label_dict[key])]

In [22]:
new_route_dict

{0: {u'desti': 1,
  u'length_of_route': 20.61412691556436,
  u'link_idx_list': [59, 54, 25, 13],
  u'node_list': [22, 16, 17, 6, 4],
  u'origi': 11},
 1: {u'desti': 1,
  u'length_of_route': 21.150250340488512,
  u'link_idx_list': [61, 25, 13],
  u'node_list': [22, 17, 6, 4],
  u'origi': 11},
 2: {u'desti': 1,
  u'length_of_route': 22.13084965216668,
  u'link_idx_list': [59, 33, 21, 13],
  u'node_list': [22, 16, 8, 6, 4],
  u'origi': 11},
 3: {u'desti': 1,
  u'length_of_route': 28.527571308662345,
  u'link_idx_list': [59, 54, 25, 9, 7],
  u'node_list': [22, 16, 17, 6, 3, 2],
  u'origi': 11},
 4: {u'desti': 1,
  u'length_of_route': 29.063694733586498,
  u'link_idx_list': [61, 25, 9, 7],
  u'node_list': [22, 17, 6, 3, 2],
  u'origi': 11},
 5: {u'desti': 1,
  u'length_of_route': 30.044294045264667,
  u'link_idx_list': [59, 33, 21, 9, 7],
  u'node_list': [22, 16, 8, 6, 3, 2],
  u'origi': 11},
 6: {u'desti': 1,
  u'length_of_route': 29.5922932335393,
  u'link_idx_list': [67, 45, 19],
  u'nod

In [23]:
with open('../temp_files/new_route_dict_journal.json', 'w') as json_file:
    json.dump(new_route_dict, json_file)

In [24]:
with open('../temp_files/new_route_dict_journal.json', 'r') as json_file:
    new_route_dict = json.load(json_file)

In [25]:
# number of routes
number_of_routes = len(new_route_dict)

In [26]:
number_of_routes

3710

In [27]:
# initialize the route_choice_probability_matrix P
P = np.zeros((number_of_od_pairs, number_of_routes))

np.shape(P)

(380, 3710)

In [28]:
for row in range(number_of_od_pairs):
    for column in range(number_of_routes):
        if new_route_dict[str(column)]['origi'] == od_pair_label_dict[row][0] and \
        new_route_dict[str(column)]['desti'] == od_pair_label_dict[row][1]:
            P[row, column] = 1

In [29]:
# for column in range(r):
#     assert(sum(P[:, column]) <= 1)

In [30]:
# for row in range(i):
#     assert(sum(P[row, :]) >= 1)

### Use sparse matrix to save memory

In [31]:
P = csr_matrix(P)

In [32]:
zdump(P, '../temp_files/OD_pair_route_incidence_journal.pkz')

In [33]:
link_label_dict = zload('../temp_files/link_label_dict_journal.pkz')
link_label_dict_ = zload('../temp_files/link_label_dict_journal_.pkz')

In [34]:
len(link_label_dict)

258

In [35]:
# number of links
number_of_links = len(link_label_dict)

In [36]:
number_of_links

258

In [37]:
# initialize the link_route incidence matrix
A = np.zeros((number_of_links, number_of_routes))

# create link_route incidence matrix A
# route index
for i in range(number_of_routes):
    # link index
    for j in range(number_of_links):
        if j in new_route_dict[str(i)]['link_idx_list']:
            A[j, i] = 1

### Use sparse matrix to save memory

In [38]:
A = csc_matrix(A)        

zdump(A, '../temp_files/link_route_incidence_matrix_journal.pkz')

In [39]:
matrix_rank(A.todense())

232

In [40]:
print(np.size(A, 0))
print(np.size(A, 1))

258
3710
